In [1]:
import requests
import pandas as pd
from tqdm import tqdm
from bs4 import BeautifulSoup
from selenium import webdriver

In [2]:
category_list = ['bath', 'bedding', 'cleaning_supplies', 
'furniture', 'heating_cooling_airquality', 'home_storage_organization', 
'irons_steamers', 'kids_home_store', 'kitchen_dining', 
'lighting_ceiling_fans', 'party_supplies', 'seasonal_decor', 
'vacuums_floor_care', 'wall_art']

In [ ]:
category = category_list[0]

In [3]:
url = f'https://www.amazon.com/s?i=specialty-aps&bbn=16225011011&rh=n%3A%2116225011011%2Cn%3A13679381&ref=nav_em__nav_desktop_sa_intl_{category}_0_2_17_10'

In [4]:
productAsin = []
productTitle = []
productPrice = []
productLink = []

chrome_driver_path = ""
driver = webdriver.Chrome(f"{chrome_driver_path}/chromedriver.exe")

for x in tqdm(range(25)):
    driver.get(url)
    source = driver.page_source
    bs = BeautifulSoup(source)
    base = "https://www.amazon.com"
    rt = [base + x['href'] for x in bs.findAll("a", {"class":"a-link-normal s-no-outline"})]
    next_page = bs.find("a", {"class" : "s-pagination-item s-pagination-next s-pagination-button s-pagination-separator"})["href"]

    for i in rt:
        try:
            driver.get(i)
        except Exception as e:
            print('WebDriverException')
            driver = webdriver.Chrome(f'{chrome_driver_path}/chromedriver.exe')
            driver.get(i)

        driver.implicitly_wait(10)
        obj = BeautifulSoup(driver.page_source, 'html.parser')   

        try:
            productTitle.append(obj.findAll('span',{'id':'productTitle'})[0].get_text().strip())
        except Exception as e:
            print('Title 오류')
            productTitle.append(None)
            
        try:
            productPrice.append(obj.findAll('span',{'class':'a-offscreen'})[0].get_text())
        except Exception as e:
            print('Price 오류')
            productPrice.append(None)
        
        try:
            productLink.append(driver.current_url)
        except Exception as e:
            print('Link 오류')
            productLink.append(None)

        try:
            try:
                idx = 0
                while idx < 30 :
                    for i in obj.findAll('th',{'class':'a-color-secondary a-size-base prodDetSectionEntry'})[idx]:
                        if i != ' ASIN ' :
                            idx += 1
                        else :
                            aa = idx
                            idx += 1
            except Exception as e:
                print(e)
                print('Asin 오류')
        
            productAsin.append(obj.findAll('td',{'class':'a-size-base prodDetAttrValue'})[aa].get_text().strip())
        except Exception as e:
            try:
                try:
                    elements = driver.find_elements_by_css_selector('#detailBullets_feature_div > ul > li')
                    for i in range(6):
                        if 'ASIN' in elements[i].get_attribute('innerHTML'):
                            asin = elements[i].get_attribute('innerHTML')[-25:-15]
                except Exception as e:
                    print(e)

                productAsin.append(asin)
            except Exception as e:
                productAsin.append(None)

    url = base + next_page
    

C:\Users\LIMDON~1\AppData\Local\Temp/ipykernel_23356/1788868855.py:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(f"{chrome_driver_path}chromedriver.exe")


WebDriverException: Message: 'chromedriver.exe' executable needs to be in PATH. Please see https://chromedriver.chromium.org/home


In [5]:
print(len(productTitle))
print(len(productAsin))
print(len(productLink))
print(len(productPrice))


0
0
0
0


In [6]:
df=pd.DataFrame({'Name':productTitle,'Asin':productAsin,'Price':productPrice,'Link':productLink})
df

,Name,Asin,Price,Link


In [7]:
df = df.dropna(subset=['Asin'])

In [8]:
df = df.drop_duplicates(['Asin'])

In [9]:
df = df[df['Price'].str.startswith('$') & ~df['Asin'].isnull()]

AttributeError: Can only use .str accessor with string values!

In [10]:
mask = (df['Asin'].str.len() == 10) & (df[df['Asin'].str.startswith('B')])
df = df.loc[mask]

AttributeError: Can only use .str accessor with string values!

In [12]:
meta_route = ""
df.to_csv(f"{route}/{category}.csv", encoding='utf-8-sig', index=False)

FileNotFoundError: [Errno 2] No such file or directory: '../data_storage/Seasonal_Decor.csv'